In [2]:
import pandas as pd
import numpy as np

# Scaffold
|Element|Your Plan|
|-|-|
|Hypothesis|MN has a systematic difference in fatal crashes involving interstate junctions than other states.|
|Outcome/Metric/Test Stat|Mean fatal junction related interstate crashes in MN per year - non-MN|
|Unit of Analysis|1 state - year |
|Data Source(s)|https://cdan.dot.gov/query|
|Why this data works|It has yearly counts by state of fatal motor vehicle crashes related to interstate junctions |
|Uncertainty Metric|Number of motor vehicle crashes by state-year |
|Null Hypothesis|Null: MN has no difference in number of fatal interstate junction motor vehicle crashes. |

In [5]:
df = pd.read_excel("CrashReport.xlsx")
df

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,National Highway Traffic Safety Administration (NHTSA) Motor Vehicle Crash Data Querying and Reporting,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Fatal Motor Vehicle Crashes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Years: 2013-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fatal Motor Vehicle Crashes 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,State \nby Interstate \nby Relation To Junction,NaN,NaN,Crash Date (Year),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,NaN,Total,Junction,7005,7098.0,7926.0,8545.0,8594.0,8361.0,8444.0,8824.0,9754.0,9966.0,9724.0,94241
910,NaN,NaN,Non-Junction,20769,20554.0,22161.0,23582.0,23169.0,22847.0,22425.0,24367.0,26674.0,25934.0,24610.0,257092
911,NaN,NaN,Other,2375,2324.0,2389.0,2567.0,2728.0,2624.0,2527.0,2642.0,3274.0,3407.0,3207.0,30064
912,NaN,NaN,Unknown,53,80.0,62.0,54.0,69.0,87.0,91.0,102.0,83.0,115.0,113.0,909
